<img src="https://www.uia.no/var/uia/storage/images/events/digital-learning-in-nordic-higher-education-before-and-after-covid-19/1997143-10-eng-GB/digital-learning-in-nordic-higher-education-before-and-after-covid-19_fullwidth.jpg">

# Importing Libraries

In [ ]:
import numpy as np 
import pandas as pd 
import os
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import missingno as msno
%matplotlib inline
import glob
import plotly.graph_objs as go
import plotly.offline as py
from plotly.subplots import make_subplots
from pandas_profiling import ProfileReport

# Data Description

What is the state of digital learning in 2020? And how does the engagement of digital learning relate to factors such as district demographics, broadband access, and state/national level policies and events?

We include three basic sets of files to help you get started. The engagement data are based on LearnPlatform’s Student Chrome Extension. The extension collects page load events of over 10K education technology products in our product library, including websites, apps, web apps, software programs, extensions, ebooks, hardwares, and services used in educational institutions. The engagement data have been aggregated at school district level, and each file represents data from one school district. The product file includes information about the characteristics of the top 372 products with most users in 2020. The district file includes information about the characteristics of school districts, including data from [National Center for Education Statistics (NCES)](https://nces.ed.gov/), [The Federal Communications Commission (FCC)](https://www.fcc.gov/), and [Edunomics Lab](https://edunomicslab.org/).

# Load Data

In [ ]:
class Preprocessor:
    def __init__(self, df):
        self.df = df.copy(deep=True)
    def missing_values(self):
        total = self.df.isnull().sum().sort_values(ascending=False)
        percent = (self.df.isnull().sum()/self.df.isnull().count()).sort_values(ascending=False)
        missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
        return missing_data.head(20)
    def msno_m(self):
        print(msno.bar(self.df));
        print(msno.matrix(self.df));
        print(msno.heatmap(self.df));
    def desc_data(self, features = None):
        if features == None:
            print(self.df.describe())
        else:
            for feat in features:
                print("--"*28)
                print(self.df[feat].describe())

## Product information data

> - The product file ```products_info.csv``` includes information about the characteristics of the top 372 products with most users in 2020. The categories listed in this file are part of LearnPlatform's product taxonomy. Some products may not have labels due to being duplicate, lack of accurate url or other reasons.

In [ ]:
products_data = pd.read_csv("../input/learnplatform-covid19-impact-on-digital-learning/products_info.csv")
products_data.head()

In [ ]:
products_data.info()

In [ ]:
preprocessor_products = Preprocessor(products_data)

In [ ]:
preprocessor_products.missing_values()

Since the rate of missing values in our data is not very high **(0.05)** and our variables are categorical, we can replace the missing values with mod values. However, we need to analyze whether the missing values are completely random or not.

In [ ]:
preprocessor_products.msno_m()

According to the nullity matrix, there is a strong nullity correlation between Sector(s) and Primary Essential Function variables. This means that if Sector variable has a missing value in one row Primary Essential Function variable will %100 has a missing value in the same row. Normally, if the missing value ratio was higher than %20 any procedure would affect our analysis, but in our case replacing with mode values or dropping null values directly would not change the result of our analysis.

In [ ]:
products_data.apply(lambda x: x.fillna(x.mode()[0], axis = 0,inplace=True))

In [ ]:
products_data.isnull().sum()

In [ ]:
report_products = ProfileReport(products_data)
report_products

## District information data

The district file districts_info.csv includes information about the characteristics of school districts, including data from [National Center for Education Statistics (NCES)](https://nces.ed.gov/), [The Federal Communications Commission (FCC)](https://www.fcc.gov/), and [Edunomics Lab](https://edunomicslab.org/).

In this data set, we removed the identifiable information about the school districts. We also used an open source tool [ARX](https://arx.deidentifier.org/) (Prasser et al. 2020) to transform several data fields and reduce the risks of re-identification. For data generalization purposes some data points are released with a range where the actual value falls under. Additionally, there are many missing data marked as 'NaN' indicating that the data was suppressed to maximize anonymization of the dataset.

In [ ]:
districts_data = pd.read_csv("../input/learnplatform-covid19-impact-on-digital-learning/districts_info.csv")
districts_data.head()

In [ ]:
districts_data.info()

In [ ]:
districts_data.drop_duplicates(inplace = True)

In [ ]:
preprocessor_districts = Preprocessor(districts_data)

In [ ]:
preprocessor_districts.missing_values()

In [ ]:
preprocessor_districts.msno_m()

According to the nullity matrix, the correlations between the variables are not **completely random**. Therefore, we will do the analysis together with the missing values.

In [ ]:
report_products = ProfileReport(districts_data)
report_products

## Engagement data

The engagement data are aggregated at school district level, and each file in the folder engagement_data represents data from one school district. The 4-digit file name represents district_id which can be used to link to district information in district_info.csv. The lp_id can be used to link to product information in product_info.csv.

In [ ]:
path = '../input/learnplatform-covid19-impact-on-digital-learning/engagement_data' 
path_ext = glob.glob(path + "/*.csv")
list_names = []
for i in path_ext:
    df = pd.read_csv(i, index_col=None, header=0)
    district_id = i.split("/")[4].split(".")[0]
    df["district_id"] = district_id
    list_names.append(df)
    
engagement_data = pd.concat(list_names)
engagement_data = engagement_data.reset_index(drop=True)

In [ ]:
engagement_data.head()

In [ ]:
engagement_data.info()

In [ ]:
engagement_data["time"] = pd.to_datetime(engagement_data["time"], format="%Y-%m-%d")
engagement_data["district_id"] = engagement_data["district_id"].astype(int)

In [ ]:
print("Engagement data contains {} unique products".format(engagement_data["lp_id"].nunique()))

In [ ]:
preprocessor_engagement = Preprocessor(engagement_data)

In [ ]:
preprocessor_engagement.missing_values()

In [ ]:
preprocessor_engagement.msno_m()

Deleting missing data (total of 5378409 rows) will cause **information loss** for variables in other rows. Therefore, we will leave it as it is.

# Data Visualization

### Products_data 

In [ ]:
products_data["PES-MC"] = products_data["Primary Essential Function"].apply(lambda x: x.split(' - ')[0])
products_data["PES-SC"] = products_data["Primary Essential Function"].apply(lambda x: x.split(' - ')[1])

In [ ]:
plt.figure(figsize=(16, 16))
sns.set(style="darkgrid")
ax = sns.countplot(x="Primary Essential Function", data=products_data, order = products_data["Primary Essential Function"].value_counts().index)
plt.title("Distribution of the basic function of the product",font="Serif", size=20)
plt.xticks(rotation=90)

As we can see from countplot, the most common main category is LC(Learning & Curriculum). In addition, digital platforms, sites, and resources are the most common way of learning among people.

In [ ]:
fig, ax = plt.subplots(5, 1, figsize=(16,16))
sns.countplot(data=products_data, x='PES-MC', palette ='Blues_d', ax=ax[0])
ax[0].set_title('Main Categories in Primary Essential Function')

for subs in np.arange(len(products_data["PES-MC"].unique())):
    sns.countplot(data=products_data[products_data["PES-MC"] == products_data["PES-MC"].unique()[subs]],
                  x='PES-SC',
                  palette ='Blues_d', ax=ax[subs+1])
    ax[subs+1].set_title('Sub-Categories of {}'.format(products_data["PES-MC"].unique()[subs]))
    ax[subs+1].set_xticklabels(ax[subs+1].get_xticklabels(), rotation=90)
    ax[subs+1].set(xlabel=None)
    plt.subplots_adjust(top = 1.99, bottom=0.11, hspace=1.5, wspace=0.9)

In [ ]:
#Sector Distribution
fig = go.Figure(data=[go.Pie(labels=products_data["Sector(s)"].value_counts().index,
                             values=products_data["Sector(s)"].value_counts().values, hole=.3)])
fig.show()

### Districts Data

In [ ]:
plt.figure(figsize=(16, 16))
sns.set(style="darkgrid")
sns.countplot(y="state",data=districts_data,order=districts_data.state.value_counts().index,
              palette="Spectral",linewidth=1, hue ="locale")
plt.title("State and Types of Areas Distribution", size=18)
plt.show()

In [ ]:
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=districts_data.state.value_counts().index, values=districts_data.state.value_counts().values, name="States",
                    )
              ,1, 1)
fig.add_trace(go.Pie(labels=districts_data.locale.value_counts().index, values=districts_data.locale.value_counts().values, name="Locale"),
              1, 2)

fig.update_traces(hole=.4, hoverinfo="label+percent+name")

fig.update_layout(
    title_text="State and Types of Areas Distribution",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='State', x=0.18, y=0.5, font_size=20, showarrow=False),
                 dict(text='Locale', x=0.82, y=0.5, font_size=20, showarrow=False)])
fig.show()

In [ ]:
ratio_names = districts_data.drop(["district_id", 'state', 'locale'], axis = 1).columns

In [ ]:
for ratios in ratio_names:
    fig = px.bar(districts_data, x= districts_data[ratios].value_counts().index,
                 y= districts_data[ratios].value_counts().values,
                 barmode='group', title = ratios)
    fig.show()

School Districts in USA

In [ ]:
#The df_usa data contains the state and the abbreviation of that state as a variable.
df_usa = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv')
us_states = {key: value for key, value in zip(df_usa["state"], df_usa["code"])}
df_copy = districts_data.copy()

df_copy['state_code'] = df_copy['state'].replace(us_states)
state_num = df_copy['state_code'].value_counts().to_frame().reset_index(drop=False)
state_num.columns = ['state_code', 'count']

fig = go.Figure(data=go.Choropleth(
    locations= state_num['state_code'], # Spatial coordinates
    z = state_num["count"], # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'sunset',
    colorbar_title = "School Districts",
))

fig.update_layout(
    title_text = 'School Districts Available in each USA States',
    geo_scope='usa', # limite map scope to USA
)

fig.show()

Now we will take a look at the mean of the ratios per state. 

In [ ]:
def interval_pct(ratio):
    if ratio == '[0, 0.2[':
        return 0.1
    elif ratio == '[0.2, 0.4[':
        return 0.3
    elif ratio == '[0.4, 0.6[':
        return 0.5
    elif ratio == '[0.6, 0.8[':
        return 0.7
    elif ratio == '[0.8, 1[':
        return 0.9
    else:
        return np.nan

def interval_total_raw(ratio):
    if ratio == '[4000, 6000[':
        return 5000
    elif ratio == '[6000, 8000[':
        return 7000
    elif ratio == '[8000, 10000[':
        return 9000
    elif ratio == '[10000, 12000[':
        return 11000
    elif ratio ==  '[12000, 14000[':
        return 13000
    elif ratio ==  '[14000, 16000[':
        return 15000
    elif ratio == '[16000, 18000[':
        return 17000
    elif ratio ==  '[18000, 20000[':
        return 19000
    elif ratio ==  '[20000, 22000[':
        return 21000
    elif ratio ==  '[22000, 24000[':
        return 21000
    else: 
        return np.nan

df_copy['pct_black_hispanic_num'] = df_copy['pct_black/hispanic'].apply(lambda x: interval_pct(x))
df_copy['pct_free_reduced_num'] = df_copy['pct_free/reduced'].apply(lambda x: interval_pct(x))
df_copy['pp_total_raw_num'] = df_copy['pp_total_raw'].apply(lambda x: interval_total_raw(x))

In [ ]:
mean_black_hispanic = df_copy.groupby('state_code')["pct_black_hispanic_num"].mean().to_frame().reset_index(drop=False)
mean_free_reduced = df_copy.groupby('state_code')["pct_free_reduced_num"].mean().to_frame().reset_index(drop=False)
mean_total_raw = df_copy.groupby('state_code')["pp_total_raw_num"].mean().to_frame().reset_index(drop=False)

ratio_dataframe = pd.merge(mean_black_hispanic,mean_free_reduced)
ratio_dataframe = pd.merge(ratio_dataframe,mean_total_raw)

for i in ratio_dataframe.drop("state_code", axis =1):
    fig = go.Figure(data=go.Choropleth(
        locations= ratio_dataframe['state_code'], # Spatial coordinates
        z = ratio_dataframe[i], # Data to be color-coded
        locationmode = 'USA-states', # set of locations match entries in `locations`
        colorscale = 'sunset',
    ))

    fig.update_layout(
        title_text = f'Mean {i} in each USA States',
        geo_scope='usa', # limite map scope to USA
    )

    fig.show()

### Engagement Data

Before we proceed, we will merge product, district, and engagement data to draw better **insight** from data.

In [ ]:
product_engagement = pd.merge(products_data, engagement_data, left_on='LP ID', right_on='lp_id')
district_engagement = pd.merge(df_copy, engagement_data, left_on='district_id', right_on='district_id')

In [ ]:
# Monthly change in access to digital platforms and student engagements.
digital_learning = product_engagement[(product_engagement["PES-SC"] == "Digital Learning Platforms") | (product_engagement["PES-SC"] == "Virtual Classroom") | (product_engagement["PES-SC"] == "Online Course Providers & Technical Skills Development")]
digital_learning.head()

In [ ]:
pct_acc_time = digital_learning.groupby(['time',"PES-SC"])["pct_access"].mean().to_frame().reset_index(drop=False)
eng_time = digital_learning.groupby(['time',"PES-SC"])["engagement_index"].mean().to_frame().reset_index(drop=False)

Those viewing these charts should remember that there are summer holidays in July and August and therefore there are no classes to attend. According to the pct_access graph at least one page-load event of a given product and on a given day by students increases over time. Although students have access, their interaction is not the same for virtual classrooms.  However, online courses for students appear to be in trend. During **Covid-19** , it cannot be ignored that online education has increased over time. The low intensity of engagement in virtual classes may indicate that students physically go to school and attend classes.

In [ ]:
fig = px.line(pct_acc_time, x= pct_acc_time.time, y=pct_acc_time.pct_access,
              color=pct_acc_time["PES-SC"])
fig.show()

In [ ]:
fig = px.line(eng_time, x= eng_time.time, y=eng_time.engagement_index,
              color=eng_time["PES-SC"])
fig.show()

In [ ]:
mean_pct_access = district_engagement.groupby('state_code')["pct_access"].mean().to_frame().reset_index(drop=False)

fig = go.Figure(data=go.Choropleth(
        locations= mean_pct_access['state_code'], # Spatial coordinates
        z = mean_pct_access["pct_access"], # Data to be color-coded
        locationmode = 'USA-states', # set of locations match entries in `locations`
        colorscale = 'sunset',
))

fig.update_layout(
    title_text = "Percentage of the students access to learning platforms in each state",
    geo_scope='usa', # limite map scope to USA
)

fig.show()

In [ ]:
mean_eng_index = district_engagement.groupby('state_code')["engagement_index"].mean().to_frame().reset_index(drop=False)

fig = go.Figure(data=go.Choropleth(
        locations= mean_eng_index['state_code'], # Spatial coordinates
        z = mean_eng_index["engagement_index"], # Data to be color-coded
        locationmode = 'USA-states', # set of locations match entries in `locations`
        colorscale = 'sunset',
))

fig.update_layout(
    title_text = "Number of the students engage learning platforms in each state",
    geo_scope='usa', # limite map scope to USA
)

fig.show()

Product names and pct_access of specific products.

In [ ]:
product_id_pct = product_engagement.groupby(["Product Name", "PES-SC"])["pct_access"].mean().sort_values(ascending=False).to_frame().reset_index(drop=False)
product_id_eng = product_engagement.groupby(["Product Name", "PES-SC"])["engagement_index"].mean().sort_values(ascending=False).to_frame().reset_index(drop=False)

In [ ]:
learning_type = ["Virtual Classroom", "Digital Learning Platforms", "Content Creation & Curation", 
                "School Management Software","Sites, Resources & Reference", "Learning Management Systems (LMS)",
                "Online Course Providers & Technical Skills Development", "Classroom Engagement & Instruction"]

In [ ]:
fig = make_subplots(rows=8, cols=2, subplot_titles=(f'Top 4 product type of {learning_type[0]}',
                                                    f'Top 4 product type of {learning_type[0]}',
                                                    f'Top 4 product type of {learning_type[1]}',
                                                    f'Top 4 product type of {learning_type[1]}',
                                                    f'Top 4 product type of {learning_type[2]}',
                                                    f'Top 4 product type of {learning_type[2]}',
                                                    f'Top 4 product type of {learning_type[3]}',
                                                    f'Top 4 product type of {learning_type[3]}',
                                                    f'Top 4 product type of {learning_type[4]}',
                                                    f'Top 4 product type of {learning_type[4]}',
                                                    f'Top 4 product type of {learning_type[5]}',
                                                    f'Top 4 product type of {learning_type[5]}',
                                                    f'Top 4 product type of {learning_type[6]}',
                                                    f'Top 4 product type of {learning_type[6]}',
                                                    f'Top 4 product type of {learning_type[7]}',
                                                    f'Top 4 product type of {learning_type[7]}',
                                                   ))
fig.add_trace(go.Bar(
y=product_id_pct[product_id_pct["PES-SC"] == learning_type[0]]["Product Name"][:4].value_counts().index,
x=product_id_pct[product_id_pct["PES-SC"] == learning_type[0]]["pct_access"][:4].values,
name='pct_access',
orientation='h',
marker=dict(
    color='rgba(63, 172, 235, 0.6)',
    line=dict(color='rgba(63, 172, 235, 1.0)', width=3)
    )
),row = 1, col = 1)


fig.add_trace(go.Bar(
y=product_id_eng[product_id_eng["PES-SC"] == learning_type[0]]["Product Name"][:4].value_counts().index,
x=product_id_eng[product_id_eng["PES-SC"] == learning_type[0]]["engagement_index"][:4].values,
name='engagement_index',
orientation='h',
marker=dict(
    color='rgba(230, 73, 64, 0.6)',
    line=dict(color='rgba(230, 73, 64, 1.0)', width=3)
    )
),row = 1, col = 2)

fig.add_trace(go.Bar(
y=product_id_pct[product_id_pct["PES-SC"] == learning_type[1]]["Product Name"][:4].value_counts().index,
x=product_id_pct[product_id_pct["PES-SC"] == learning_type[1]]["pct_access"][:4].values,
name='pct_access',
orientation='h',
marker=dict(
    color='rgba(63, 172, 235, 0.6)',
    line=dict(color='rgba(63, 172, 235, 1.0)', width=3)
    )
),row = 2, col = 1)


fig.add_trace(go.Bar(
y=product_id_eng[product_id_eng["PES-SC"] == learning_type[1]]["Product Name"][:4].value_counts().index,
x=product_id_eng[product_id_eng["PES-SC"] == learning_type[1]]["engagement_index"][:4].values,
name='engagement_index',
orientation='h',
marker=dict(
    color='rgba(230, 73, 64, 0.6)',
    line=dict(color='rgba(230, 73, 64, 1.0)', width=3)
    )
),row = 2, col = 2)





fig.add_trace(go.Bar(
y=product_id_pct[product_id_pct["PES-SC"] == learning_type[2]]["Product Name"][:4].value_counts().index,
x=product_id_pct[product_id_pct["PES-SC"] == learning_type[2]]["pct_access"][:4].values,
name='pct_access',
orientation='h',
marker=dict(
    color='rgba(63, 172, 235, 0.6)',
    line=dict(color='rgba(63, 172, 235, 1.0)', width=3)
    )
),row = 3, col = 1)


fig.add_trace(go.Bar(
y=product_id_eng[product_id_eng["PES-SC"] == learning_type[2]]["Product Name"][:4].value_counts().index,
x=product_id_eng[product_id_eng["PES-SC"] == learning_type[2]]["engagement_index"][:4].values,
name='engagement_index',
orientation='h',
marker=dict(
    color='rgba(230, 73, 64, 0.6)',
    line=dict(color='rgba(230, 73, 64, 1.0)', width=3)
    )
),row = 3, col = 2)



fig.add_trace(go.Bar(
y=product_id_pct[product_id_pct["PES-SC"] == learning_type[3]]["Product Name"][:4].value_counts().index,
x=product_id_pct[product_id_pct["PES-SC"] == learning_type[3]]["pct_access"][:4].values,
name='pct_access',
orientation='h',
marker=dict(
    color='rgba(63, 172, 235, 0.6)',
    line=dict(color='rgba(63, 172, 235, 1.0)', width=3)
    )
),row = 4, col = 1)


fig.add_trace(go.Bar(
y=product_id_eng[product_id_eng["PES-SC"] == learning_type[3]]["Product Name"][:4].value_counts().index,
x=product_id_eng[product_id_eng["PES-SC"] == learning_type[3]]["engagement_index"][:4].values,
name='engagement_index',
orientation='h',
marker=dict(
    color='rgba(230, 73, 64, 0.6)',
    line=dict(color='rgba(230, 73, 64, 1.0)', width=3)
    )
),row = 4, col = 2)


fig.add_trace(go.Bar(
y=product_id_pct[product_id_pct["PES-SC"] == learning_type[4]]["Product Name"][:4].value_counts().index,
x=product_id_pct[product_id_pct["PES-SC"] == learning_type[4]]["pct_access"][:4].values,
name='pct_access',
orientation='h',
marker=dict(
    color='rgba(63, 172, 235, 0.6)',
    line=dict(color='rgba(63, 172, 235, 1.0)', width=3)
    )
),row = 5, col = 1)


fig.add_trace(go.Bar(
y=product_id_eng[product_id_eng["PES-SC"] == learning_type[4]]["Product Name"][:4].value_counts().index,
x=product_id_eng[product_id_eng["PES-SC"] == learning_type[4]]["engagement_index"][:4].values,
name='engagement_index',
orientation='h',
marker=dict(
    color='rgba(230, 73, 64, 0.6)',
    line=dict(color='rgba(230, 73, 64, 1.0)', width=3)
    )
),row = 5, col = 2)


fig.add_trace(go.Bar(
y=product_id_pct[product_id_pct["PES-SC"] == learning_type[5]]["Product Name"][:4].value_counts().index,
x=product_id_pct[product_id_pct["PES-SC"] == learning_type[5]]["pct_access"][:4].values,
name='pct_access',
orientation='h',
marker=dict(
    color='rgba(63, 172, 235, 0.6)',
    line=dict(color='rgba(63, 172, 235, 1.0)', width=3)
    )
),row = 6, col = 1)


fig.add_trace(go.Bar(
y=product_id_eng[product_id_eng["PES-SC"] == learning_type[5]]["Product Name"][:4].value_counts().index,
x=product_id_eng[product_id_eng["PES-SC"] == learning_type[5]]["engagement_index"][:4].values,
name='engagement_index',
orientation='h',
marker=dict(
    color='rgba(230, 73, 64, 0.6)',
    line=dict(color='rgba(230, 73, 64, 1.0)', width=3)
    )
),row = 6, col = 2)


fig.add_trace(go.Bar(
y=product_id_pct[product_id_pct["PES-SC"] == learning_type[6]]["Product Name"][:4].value_counts().index,
x=product_id_pct[product_id_pct["PES-SC"] == learning_type[6]]["pct_access"][:4].values,
name='pct_access',
orientation='h',
marker=dict(
    color='rgba(63, 172, 235, 0.6)',
    line=dict(color='rgba(63, 172, 235, 1.0)', width=3)
    )
),row = 7, col = 1)


fig.add_trace(go.Bar(
y=product_id_eng[product_id_eng["PES-SC"] == learning_type[6]]["Product Name"][:4].value_counts().index,
x=product_id_eng[product_id_eng["PES-SC"] == learning_type[6]]["engagement_index"][:4].values,
name='engagement_index',
orientation='h',
marker=dict(
    color='rgba(230, 73, 64, 0.6)',
    line=dict(color='rgba(230, 73, 64, 1.0)', width=3)
    )
),row = 7, col = 2)


fig.add_trace(go.Bar(
y=product_id_pct[product_id_pct["PES-SC"] == learning_type[7]]["Product Name"][:4].value_counts().index,
x=product_id_pct[product_id_pct["PES-SC"] == learning_type[7]]["pct_access"][:4].values,
name='pct_access',
orientation='h',
marker=dict(
    color='rgba(63, 172, 235, 0.6)',
    line=dict(color='rgba(63, 172, 235, 1.0)', width=3)
    )
),row = 8, col = 1)


fig.add_trace(go.Bar(
y=product_id_eng[product_id_eng["PES-SC"] == learning_type[7]]["Product Name"][:4].value_counts().index,
x=product_id_eng[product_id_eng["PES-SC"] == learning_type[7]]["engagement_index"][:4].values,
name='engagement_index',
orientation='h',
marker=dict(
    color='rgba(230, 73, 64, 0.6)',
    line=dict(color='rgba(230, 73, 64, 1.0)', width=3)
    )
),row = 8, col = 2)

fig.update_layout(height=1800, width=1800)
fig.show()